In [2]:
import requests as r
from fuzzywuzzy import process
import pprint
pp = pprint.PrettyPrinter(indent=2)

/Users/adam/Documents/vectors/venv/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
response = r.get('https://celestrak.org/NORAD/elements/gp.php?GROUP=active&FORMAT=tle', verify=True)
response.status_code

# https://celestrak.org/columns/v04n03/
# TLE formats are from this issue of Celestrak's newsletter

200

In [7]:
# parse TLEs into a dictionary and index
TLE_lines =[line.strip() for line in response.text.split('\n')]
TLE_lines

len(TLE_lines)
if TLE_lines[-1] == '':
    TLE_lines.pop()

while len(TLE_lines) % 3 != 0:
    TLE_lines.pop()
TLEs = {TLE_lines[i]: [TLE_lines[i+1], TLE_lines[i+2]] for i in range(0, len(TLE_lines), 3)}
index = list(TLEs.keys())

# fuzzy search for a satellite name
def find_satellite(name):

    match = process.extractOne(name, index)
    
    return {
        'query': name,
        'match': match[0],
        'score': match[1],
        'tle': TLEs[match[0]]
    }

find_satellite('NORSAT')

{'query': 'NORSAT',
 'match': 'NORSAT 1',
 'score': 95,
 'tle': ['1 42826U 17042B   25014.49553221  .00008953  00000+0  64024-3 0  9992',
  '2 42826  97.4053 193.4589 0009857  69.1836 291.0446 15.04634661408973']}

In [9]:
def parse_tle_line1(line):
    # Parse each field according to the TLE format specification
    data = {
        'line_number': line[0:1],          # Field 1.1: Line Number
        'satellite_number': line[2:7],      # Field 1.2: Satellite Number
        'classification': line[7],          # Field 1.3: Classification
        'launch_year': line[9:11],         # Field 1.4: Launch Year
        'launch_number': line[11:14],      # Field 1.5: Launch number of the year
        'launch_piece': line[14:17],       # Field 1.6: Piece of launch
        'epoch_year': line[18:20],         # Field 1.7: Epoch Year
        'epoch': line[20:32],              # Field 1.8: Epoch Day and fractional
        'first_derivative': line[33:43],    # Field 1.9: First Time Derivative
        'second_derivative': line[44:52],   # Field 1.10: Second Time Derivative
        'bstar': line[53:61],              # Field 1.11: BSTAR drag term
        'ephemeris_type': line[62],        # Field 1.12: Ephemeris type
        'element_number': line[64:68],      # Field 1.13: Element number
        'checksum': line[68]               # Field 1.14: Checksum
    }
    
    return data

def parse_tle_line2(line):
    data = {
        'line_number': line[0:1],          # Field 2.1: Line Number
        'satellite_number': line[2:7],      # Field 2.2: Satellite Number
        'inclination': line[8:16],         # Field 2.3: Inclination
        'right_ascension': line[17:25],    # Field 2.4: Right Ascension
        'eccentricity': line[26:33],       # Field 2.5: Eccentricity
        'argument_of_perigee': line[34:42], # Field 2.6: Argument of Perigee
        'mean_anomaly': line[43:51],       # Field 2.7: Mean Anomaly
        'mean_motion': line[52:63],        # Field 2.8: Mean Motion
        'revolution_number': line[63:68],   # Field 2.9: Revolution Number
        'checksum': line[68]               # Field 2.10: Checksum
    }
    return data

# parse_tle_line2(match['tle'][1])

match = find_satellite('ZARYA')
print(match['match'])
pp.pprint(parse_tle_line1(match['tle'][0]))
pp.pprint(parse_tle_line2(match['tle'][1]))


ISS (ZARYA)
{ 'bstar': ' 27291-3',
  'checksum': '7',
  'classification': 'U',
  'element_number': ' 999',
  'ephemeris_type': '0',
  'epoch': '015.13687281',
  'epoch_year': '25',
  'first_derivative': ' .00015162',
  'launch_number': '067',
  'launch_piece': 'A  ',
  'launch_year': '98',
  'line_number': '1',
  'satellite_number': '25544',
  'second_derivative': ' 00000+0'}
{ 'argument_of_perigee': ' 98.9982',
  'checksum': '4',
  'eccentricity': '0002338',
  'inclination': ' 51.6394',
  'line_number': '2',
  'mean_anomaly': '351.9829',
  'mean_motion': '15.50022714',
  'revolution_number': '49142',
  'right_ascension': '345.1673',
  'satellite_number': '25544'}
